In [32]:
import docplex
from docplex.mp.model import Model
import cplex

# Estruturas de dados utilizadas

## Representação geral do problema

In [33]:
class Job:
    def __init__(self, id, processing_time, size):
        self.id = id
        self.processing_time = processing_time
        self.size = size

class Machine:
    def __init__(self, id, capacity):
        self.id = id
        self.capacity = capacity

class Problem:
    def __init__(self):
        self.jobs = []
        self.machines = []
        self.model = Model("Job Scheduling")

    def add_job(self, job):
        self.jobs.append(job)

    def add_machine(self, machine):
        self.machines.append(machine)
    
    def get_number_of_jobs(self):
        return len(self.jobs)
    
    def get_number_of_machines(self):
        return len(self.machines)   
    
    def get_job(self, index):
        return self.jobs[index]
    
    def get_machine(self, index):
        return self.machines[index]


## Representação do problema específico da representação por fluxo

In [34]:
class ProblemFlow:
    def __init__(self, problem):
        self.problem = problem
        self.distinct_sizes = sorted(set(job.size for job in problem.jobs))
        self.distict_processing_times = sorted(set(job.processing_time for job in problem.jobs))
        self.jobs_by_size = {}
        for size in self.distinct_sizes:
            self.jobs_by_size[size] = [job for job in problem.jobs if job.size == size]
        
        self.vertices = sorted(list(self.distinct_sizes))
        
        self.Aj = [(i,j) 
                    for i in range(problem.get_number_of_jobs()) 
                    for j in range(problem.get_number_of_jobs()) 
                    if (i < j) and (problem.get_job(i).size == j - i)
                ]
        

    def get_number_of_distinct_sizes(self):
        return len(self.distinct_sizes)
    
    def get_number_of_distinct_processing_times(self):
        return len(self.distict_processing_times)
    
    def jobs_by_size(self, size):
        return self.jobs_by_size.get(size, [])
    
    


In [35]:
# Função que gera um problema dummy:
def get_dummy():
    problem = Problem()
    problem.add_machine(Machine(0, 10))
    problem.add_machine(Machine(1, 15))
    problem.add_job(Job(0, 5, 3))
    problem.add_job(Job(1, 7, 5))
    problem.add_job(Job(2, 4, 3))
    problem.add_job(Job(3, 6, 7))
    problem.add_job(Job(4, 8, 5))

    problem_flow = ProblemFlow(problem)
    return problem_flow

# Modelo de Fluxo (Trindade et al)

In [36]:
meu_problema = get_dummy()

In [37]:
# Criação do modelo

model = Model(name="FlowTrindade", log_output=True)

# Definição de variáveis

# Função objetivo
FO = model.FO = model.continuous_var(name="FO")

number_of_jobs = meu_problema.problem.get_number_of_jobs()
number_of_ptimes = meu_problema.get_number_of_distinct_processing_times()
fijt = model.fijt = model.binary_var_cube(
    number_of_jobs, 
    number_of_jobs, 
    number_of_ptimes, name="fijt")

yij = model.yij = model.binary_var_matrix(
    number_of_jobs,
    number_of_ptimes, name="yij")

vt = model.vt = model.binary_var_list(number_of_ptimes, name="vt")

zlt = model.zlt = model.binary_var_matrix(
    meu_problema.get_number_of_distinct_sizes(),
    number_of_ptimes, name="zlt")


In [38]:
# Restrições:

## FO
model.minimize(FO)

model.add_constraint(FO == model.sum(
    meu_problema.distict_processing_times[t] * vt[t] 
    for t in range(number_of_ptimes)
    ))
            
## Eq 18
model.add_constraints(
    model.sum(fijt[0,j,t] for j in range(1, number_of_jobs)) - vt[t] == 0
    for t in range(number_of_ptimes)
)   

[docplex.mp.LinearConstraint[](fijt_0_1_0+fijt_0_2_0+fijt_0_3_0+fijt_0_4_0-vt_0,EQ,0),
 docplex.mp.LinearConstraint[](fijt_0_1_1+fijt_0_2_1+fijt_0_3_1+fijt_0_4_1-vt_1,EQ,0),
 docplex.mp.LinearConstraint[](fijt_0_1_2+fijt_0_2_2+fijt_0_3_2+fijt_0_4_2-vt_2,EQ,0),
 docplex.mp.LinearConstraint[](fijt_0_1_3+fijt_0_2_3+fijt_0_3_3+fijt_0_4_3-vt_3,EQ,0),
 docplex.mp.LinearConstraint[](fijt_0_1_4+fijt_0_2_4+fijt_0_3_4+fijt_0_4_4-vt_4,EQ,0)]

In [39]:
model.solve()

DOcplexException: Cannot solve model: no CPLEX runtime found.